# Using Tensorflow library for Airfoil use case

In [1]:
import os
from lips import get_root_path
from lips.dataset.airfransDataSet import download_data
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

/home/milad.leyli-abadi/.conda/envs/env-lips-irt/lib/python3.8/site-packages/grid2op/Backend/PandaPowerBackend.py:31: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	/home/milad.leyli-abadi/.conda/envs/env-lips-irt/bin/python -m pip install numba

  warnings.warn(


In [35]:
# Use a GPU
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

In [36]:
# see the list of selected devices
tf.config.experimental.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

In [4]:
BENCH_CONFIG_PATH = LIPS_PATH + os.path.join("..","configurations","airfoil","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
#SIM_CONFIG_PATH = LIPS_PATH + os.path.join("..","configurations","airfoil","simulators","torch_fc.ini") #Configuration file related to the simulator
SIM_CONFIG_PATH = LIPS_PATH + os.path.join("..","configurations","airfoil","simulators","tf_fc.ini") #Configuration file related to the simulator

In [7]:
benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path = LOG_PATH)
benchmark.load(path=DIRECTORY_NAME)

Loading dataset (task: reynolds, split: test): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 496/496 [01:56<00:00,  4.24it/s]


In [8]:
print("Input attributes (features): ", benchmark.config.get_option("attr_x"))
print("Output attributes (targets): ", benchmark.config.get_option("attr_y"))

Input attributes (features):  ('x-position', 'y-position', 'x-inlet_velocity', 'y-inlet_velocity', 'distance_function', 'x-normals', 'y-normals')
Output attributes (targets):  ('x-velocity', 'y-velocity', 'pressure', 'turbulent_viscosity')


In [12]:
for attr_name in benchmark.config.get_option("attr_x"):
    print(f'{attr_name}', benchmark.train_dataset.data[attr_name].shape)

x-position (18515415,)
y-position (18515415,)
x-inlet_velocity (18515415,)
y-inlet_velocity (18515415,)
distance_function (18515415,)
x-normals (18515415,)
y-normals (18515415,)


In [13]:
for attr_name in benchmark.config.get_option("attr_y"):
    print(f'{attr_name}', benchmark.train_dataset.data[attr_name].shape)

x-velocity (18515415,)
y-velocity (18515415,)
pressure (18515415,)
turbulent_viscosity (18515415,)


In [ ]:
from lips.augmented_simulators.torch_models.fully_connected import TorchFullyConnected
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative


chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

augmented_simulator = TorchSimulator(name="torch_fc",
                                     model=TorchFullyConnected,
                                     scaler=StandardScalerIterative,
                                     log_path="log_benchmark",
                                     device="cuda:0",
                                     bench_config_path=BENCH_CONFIG_PATH,
                                     bench_config_name=BENCHMARK_NAME,
                                     sim_config_path=SIM_CONFIG_PATH,
                                     sim_config_name="DEFAULT",
                                     scalerParams=scalerParams
                                    )

In [31]:
import importlib
import lips
import lips.augmented_simulators.tensorflow_models.airfoil.fully_connected
importlib.reload(lips.augmented_simulators)
importlib.reload(lips.augmented_simulators.tensorflow_models.airfoil.fully_connected)

<module 'lips.augmented_simulators.tensorflow_models.airfoil.fully_connected' from '/home/milad.leyli-abadi/LIPS_IRT/lips/augmented_simulators/tensorflow_models/airfoil/fully_connected.py'>

In [32]:
from lips.augmented_simulators.tensorflow_models.airfoil.fully_connected import TfFullyConnectedAirfoil
from lips.dataset.scaler import StandardScaler

In [33]:
tf_fc = TfFullyConnectedAirfoil(name="tf_fc",
                                bench_config_path=BENCH_CONFIG_PATH,
                                bench_config_name=BENCHMARK_NAME,
                                sim_config_path=SIM_CONFIG_PATH,
                                sim_config_name="DEFAULT",
                                scaler=StandardScaler,
                                log_path=LOG_PATH)

In [34]:
tf_fc.train(train_dataset=benchmark.train_dataset,
            #val_dataset=benchmark.val_dataset,
            epochs=2
           )

2023-12-06 10:27:08.088560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-12-06 10:27:08.088612: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-06 10:27:08.089573: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2
 14080/144652 [=>............................] - ETA: 14:19 - loss: 0.1120 - mae: 0.1421

KeyboardInterrupt: 

In [ ]:
print("Number of parameters :", tf_fc.count_parameters())

In [ ]:
tf_fc.summary()

In [ ]:
tf_fc.plot_model(path=".")

In [ ]:
tf_fc.visualize_convergence()

In [ ]:
# TODO: log the losses
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=128,
                                              dataset="all",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )